<a href="https://colab.research.google.com/github/Rhqo/LLM_Study/blob/main/fine_tuning_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Load Amazon review datasets

In [ ]:
!pip install datasets -q
# !pip install pyarrow -q
!pip install pandas -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.


In [ ]:
from datasets import load_dataset
import pandas as pd

In [ ]:
dataset = load_dataset("imdb")
training_df = pd.DataFrame(dataset['train'])
validation_df = pd.DataFrame(dataset['test'])

In [ ]:
print(dataset)
print(training_df)
print(validation_df)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})
                                                    text  label
0      I rented I AM CURIOUS-YELLOW from my video sto...      0
1      "I Am Curious: Yellow" is a risible and preten...      0
2      If only to avoid making this type of film in t...      0
3      This film was probably inspired by Godard's Ma...      0
4      Oh, brother...after hearing about this ridicul...      0
...                                                  ...    ...
24995  A hit at the time but now better categorised a...      1
24996  I love this movie like no other. Another time ...      1
24997  This film and it's sequel Barry Mckenzie holds...      1
24998  'The Adventures Of Barry McKenzie' started lif...      1
24999 

In [ ]:
def prepare_df_for_openai(df):
  df['prompt'] = df['text']
  df['completion'] = df['label']
  df.drop_duplicates(subset=['prompt'], inplace=True)

  return df[['prompt', 'completion']].sample(len(df))

In [ ]:
training_df = prepare_df_for_openai(training_df)
validation_df = prepare_df_for_openai(validation_df)

In [ ]:
training_df.to_json('./train.jsonl', orient='records', lines=True)
training_df.to_json('./test.jsonl', orient='records', lines=True)

### Setting OpenAI CLI

In [ ]:
!pip install openai -q
!pip install jq -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.6/657.6 kB 11.5 MB/s eta 0:00:00


In [ ]:
!openai tools fine_tunes.prepare_data -f ./train.jsonl
!openai tools fine_tunes.prepare_data -f ./test.jsonl

Analyzing...

- Your file contains 24904 prompt-completion pairs
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- There are 3 examples that are very long. These are rows: [19298, 19327, 19521]
For conditional generation, and for classification the examples shouldn't be longer than 2048 tokens.
- Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tuned model where the completion should begin. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples. If you intend to do open-ended generation, then you should leave the prompts empty
- The completion should start wi

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-..."
client = OpenAI()

In [ ]:
!openai api files.create -f ./train.jsonl -p fine-tune
!openai api files.create -f ./test.jsonl -p fine-tune

Upload progress: 100% 34.0M/34.0M [00:00<00:00, 91.7Mit/s]
{
  "id": "file-zfmfiMl9qmSuRASGR2bicdcI",
  "bytes": 33957395,
  "created_at": 1719376103,
  "filename": "./train.jsonl",
  "object": "file",
  "purpose": "fine-tune",
  "status": "processed",
  "status_details": null
}
Upload progress: 100% 34.0M/34.0M [00:00<00:00, 105Mit/s]
{
  "id": "file-p3t98LbCjGNZx4cEEu1SpAmv",
  "bytes": 33957395,
  "created_at": 1719376106,
  "filename": "./test.jsonl",
  "object": "file",
  "purpose": "fine-tune",
  "status": "processed",
  "status_details": null
}


In [ ]:
train_file = client.files.create(file=open("train.jsonl", "rb"), purpose='fine-tune')
train_file_id = train_file.id
test_file = client.files.create(file=open("test.jsonl", "rb"), purpose='fine-tune')
test_file_id = test_file.id

In [ ]:
# !openai api completions.create \
# -t ./train.jsonl \
# --compute_classification_metrics \
# --classification_n_classes 2 \
# -m ada \
# --n_epochs 1

fine_tune_response = client.fine_tuning.jobs.create(
    training_file=train_file_id,
    validation_file=test_file_id,
    model="gpt-3.5-turbo",
    hyperparameters={
      "n_epochs":1,
      # compute_classification_metrics:True,
      # classification_n_classes:2
    }
)

In [ ]:
print(fine_tune_response)

FineTuningJob(id='ftjob-FLQroqtZQClWwdwdvjMnwfi0', created_at=1719376206, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=1, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-IGE1n4VtYEKHaRRX86nVPpRS', result_files=[], seed=788634160, status='validating_files', trained_tokens=None, training_file='file-VYY9JT3jJ39IJI0v2WjxerCm', validation_file='file-0mHZPQw7Fd1rQYSyO3OfnSaI', estimated_finish=None, integrations=[], user_provided_suffix=None)


### Get token probability

In [ ]:
import math

In [ ]:
prompt = training_df['prompt'].sample(1).iloc[0]

In [ ]:
res = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{"role": "user", "content": prompt}],
    max_tokens=1,
    temperature=0,
    logprobs=True
)

In [ ]:
# probs = []
# logprobs = res ['choices'][0]['logprobs']['top_logprobs']

# for logprob in logprobs:
#   _probs={}
#   for key, value in logprob.items():
#     _probs[key] = math.exp(value)
#   probs.append(_probs)

# pred = res['choices'][0].text.strip()

# print("Prompt: \n", prompt[:-1])
# print("Predicted Category:", pred)
# print("Probabilities:")
# for prob in probs:
#     for key, value in sorted(prob.items(), key=lambda x: x[1], reverse=True):
#         print(f"{key}: {value:.4f}")
#     print()

TypeError: 'ChatCompletion' object is not subscriptable

In [ ]:
# Assuming 'client' is already defined and you have a successful API response
# Replace with your actual API call if needed
# res = client.chat.completions.create(...)

# Check if the 'choices' key exists in the response
if 'choices' in res and res['choices']:
    # Access the first choice
    choice = res['choices'][0]

    # Check if 'logprobs' and 'top_logprobs' keys exist within the choice
    if 'logprobs' in choice and 'top_logprobs' in choice['logprobs']:
        logprobs = choice['logprobs']['top_logprobs']
        probs = []

        for logprob in logprobs:
            _probs = {}
            for key, value in logprob.items():
                _probs[key] = math.exp(value)
            probs.append(_probs)

        # Assuming 'text' key exists in the choice
        pred = choice.get('text', '').strip()

        print("Prompt: \n", prompt[:-1])
        print("Predicted Category:", pred)
        print("Probabilities:")
        for prob in probs:
            for key, value in sorted(prob.items(), key=lambda x: x[1], reverse=True):
                print(f"{key}: {value:.4f}")
            print()
    else:
        print("Error: 'logprobs' or 'top_logprobs' not found in the API response.")
else:
    print("Error: 'choices' not found in the API response.")

Error: 'choices' not found in the API response.
